解压数据集

In [1]:
%cd /home/aistudio/data/data100477
!unzip ./常规赛：PALM眼底彩照中黄斑中央凹定位.zip
!mv -f /home/aistudio/data/data100477/常规赛：PALM眼底彩照中黄斑中央凹定位/* /home/aistudio/data

进入工作区间

In [2]:
%cd /home/aistudio/work

/home/aistudio/work


导入包，设置随机种子。

In [3]:
import os
import math
import numpy as np
import pandas as pd
from PIL import Image
np.random.seed(0)

生成paddle数据集类
- 对图片的预处理如下：
    - resize至[224,224,3]
    - 转换为np.array
    - 除255.0使得像素分布在0~1之间
    - 改变通道位置Channel Last->Channel First
- 对标签的预处理如下：
    - 将标签坐标除以对应维度的分辨率，得到相对坐标
    - 将相对坐标缩放到新的分辨率224*224下

测试集只构建测试数据集类

In [4]:
from paddle.io import Dataset
import paddle.vision.transforms as T

# class PALMTrData(Dataset):
#     def __init__(self,path,pic_list,y):
#         super(PALMTrData,self).__init__()
#         self.path=path
#         self.y=y
#         self.pic_list=pic_list
#         print(len(self.pic_list))
    
#     def __getitem__(self,idx):
#         # X-feature
#         pic_name=self.pic_list[idx]
#         pic=Image.open(self.path+'/'+pic_name)
#         pic_size=pic.size
#         pic=pic.resize((224,224),Image.ANTIALIAS)
#         pic=np.array(pic,dtype=np.float32)
#         pic=pic/255.0
#         pic=T.to_tensor(pic)
#         # Y-label
#         label=self.y[idx].copy()
#         label[0]=224.0*label[0]/pic_size[0]
#         label[1]=224.0*label[1]/pic_size[1]
#         return pic,label

#     def __len__(self):
#         return len(self.pic_list)

class PALMTsData(Dataset):
    def __init__(self,path):
        super(PALMTsData,self).__init__()
        self.path=path
        self.pic_list=os.listdir(path)
        print(len(self.pic_list))
    
    def __getitem__(self,idx):
        pic_name=self.pic_list[idx]
        pic=Image.open(self.path+'/'+pic_name)
        pic=pic.resize((224,224),Image.ANTIALIAS)
        pic=np.array(pic,dtype=np.float32)
        pic=pic/255.0
        pic=T.to_tensor(pic)
        return pic,0

    def __len__(self):
        return len(self.pic_list)

# tr_path='../data/Train/fundus_image'
# trData=PALMTrData(path=tr_path,pic_list=trX_list,y=trY)
ts_path='../data/PALM-Testing400-Images'
tsData=PALMTsData(path=ts_path)

400


构建模型，设置随机种子，设置训练设备为GPU
- 模型使用预训练模型resnet50为主体
- 下游任务分类器结构为2层全连接网络(1000->100->2)，以relu激活，并辅以Dropout=0.2防止过拟合
- 下游任务被设定为回归任务，分别回归x,y坐标

In [5]:
import paddle
paddle.seed(0)
paddle.set_device('gpu')
from paddle import nn
import paddle.nn.functional as F
from paddle.vision.models import resnet50

class PALM(nn.Layer):
    def __init__(self):
        super(PALM,self).__init__()
        self.resnet=resnet50(pretrained=True)
        self.do1=nn.Dropout(p=0.2)
        self.lr1=nn.Linear(in_features=1000, out_features=100)
        self.do2=nn.Dropout(p=0.2)
        self.lr2=nn.Linear(in_features=100, out_features=2)

    def forward(self,x):
        x=self.resnet(x)
        x=F.relu(x)
        x=self.do1(x)
        x=self.lr1(x)
        x=F.relu(x)
        x=self.do2(x)
        x=self.lr2(x)
        return x

全局参数设置
- 集成模型数：10
- 训练、预测批大小：16
- 训练回合：50
- 学习率：固定5e-5

In [6]:
bagging=10
batch_size=16
epochs=50
lr=5e-5

逐个加载集成模型
- 优化器：adam
- Loss：与比赛的最终评价分数相同，欧氏距离=MSELoss

In [7]:
from paddle.optimizer import Adam
from paddle.nn import MSELoss
from paddle.metric import Accuracy

inputs=paddle.static.InputSpec([-1, 3, 224, 224], dtype='float32', name='input')
label=paddle.static.InputSpec([-1, 2], dtype='float32', name='label')
model=[]
for i in range(bagging):
    print('------------start load model {}------------'.format(i))
    palm=PALM()
    palm.set_state_dict(paddle.load('./savepoint/final_{}'.format(i)))
    model.append(paddle.Model(palm, inputs, label))
    model[i].prepare(optimizer=Adam(learning_rate=lr,parameters=model[i].parameters()),loss=MSELoss(),metrics=Accuracy())
    print('-------------end load model {}-------------'.format(i))

------------start load model 0------------


100%|██████████| 151272/151272 [00:02<00:00, 53272.87it/s]
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/backward.py:1666: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return list(x) if isinstance(x, collections.Sequence) else [x]


-------------end load model 0-------------
------------start load model 1------------
-------------end load model 1-------------
------------start load model 2------------
-------------end load model 2-------------
------------start load model 3------------
-------------end load model 3-------------
------------start load model 4------------
-------------end load model 4-------------
------------start load model 5------------
-------------end load model 5-------------
------------start load model 6------------
-------------end load model 6-------------
------------start load model 7------------
-------------end load model 7-------------
------------start load model 8------------
-------------end load model 8-------------
------------start load model 9------------
-------------end load model 9-------------


测试数据生成预测结构
- 集成模型结果逐个汇总
- 预测数据异步读取：2个进程

In [8]:
preds=np.zeros((bagging,400,2))
for i in range(bagging):
    pps=model[i].predict(tsData,batch_size=batch_size,num_workers=2)
    preds[i,:,:]=np.squeeze(np.array(pps)).reshape((400,2))

Predict begin...
step 25/25 [==============================] - 879ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 855ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 857ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 896ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 854ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 861ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 862ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 882ms/step        
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 867ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 87

合并集成模型结果
- 对(x,y)分别取均值
- 将预测的标签缩放回原分辨率
- 保存成提交文件

In [9]:
pred=np.mean(preds,axis=0)
pred_list=os.listdir(ts_path)
print(pred.shape)
for i,pic_name in enumerate(pred_list):
    pic=Image.open(ts_path+'/'+pic_name)
    pic_size=pic.size
    pred[i,0]=pred[i,0]*pic_size[0]/224.0
    pred[i,1]=pred[i,1]*pic_size[1]/224.0
ans=pd.DataFrame({'FileName':pred_list,'Fovea_X':pred[:,0],'Fovea_Y':pred[:,1]})
ans=ans.sort_values(by='FileName',ignore_index=True)
ans.to_csv('./Fovea_Localization_Results_Test.csv',index=0)

(400, 2)


## 注意事项与特殊说明
由于GitHub对大文件上传限制，模型训练参数checkpoint无法上传。
- 完整repo详见AI studio项目：https://aistudio.baidu.com/aistudio/projectdetail/2310885